<a href="https://colab.research.google.com/github/profliuhao/CSIT553/blob/main/CSIT553_class10_InteractiveVis_exercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CSIT 553 class 10 Exercise: Interactive Visualization


In [1]:
!pip install dash

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 20.0 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
pd.options.display.max_rows = 10

import matplotlib.pyplot as plt
import seaborn as sns

import plotly.express as px
import dash
from dash import html
from dash import dcc
from dash.dependencies import Input, Output
from dash import Dash

## About the Data Set: Chicago Food Inspections
Link: https://data.cityofchicago.org/Health-Human-Services/Food-Inspections/4ijn-s7e5

This information is derived from inspections of restaurants and other food establishments in Chicago from January 1, 2010 to the present. Inspections are performed by staff from the Chicago Department of Public Health’s Food Protection Program using a standardized procedure. The results of the inspection are inputted into a database, then reviewed and approved by a State of Illinois Licensed Environmental Health Practitioner (LEHP). For descriptions of the data elements included in this set, go to http://bit.ly/tS9IE8

Note about 7/1/2018 change to food inspection procedures that affects the data in this dataset: http://bit.ly/2yWd2JB

Disclaimer: Attempts have been made to minimize any and all duplicate inspection reports. However, the dataset may still contain such duplicates and the appropriate precautions should be exercised when viewing or analyzing these data. The result of the inspections (pass, pass with conditions or fail) as well as the violations noted are based on the findings identified and reported by the inspector at the time of the inspection, and may not reflect the findings noted at other times. For more information about Food Inspections, go to https://www.cityofchicago.org/city/en/depts/cdph/provdrs/healthy_restaurants/svcs/food-protection-services.html.

Columns in this Dataset

| Column Name     | Description       | Type       |
|-----------------|-------------------|------------|
| Inspection ID   |                   | Number     |
| DBA Name        | Doing Business As | Plain Text |
| AKA Name        | Also Known As     | Plain Text |
| License #       |                   | Number     |
| Facility Type   |                   | Plain Text |
| Risk            |                   | Plain Text |
| Address         |                   | Plain Text |
| City            |                   | Plain Text |
| State           |                   | Plain Text |
| Zip             |                   | Number     |
| Inspection Date |                   | Date&Time  |
| Results         |                   | Plain Text |
| Violations      |                   | Plain Text |
| Latitude        |                   | Number     |
| Longitude       |                   | Number     |
| Location        |                   | Location   |

## Data Preparation

In [3]:
import requests
import csv


# API endpoint URL
url = "https://data.cityofchicago.org/resource/4ijn-s7e5.json"

# Total number of rows to fetch
total_rows = 20000

# Number of rows to fetch per request
rows_per_request = 1000

# Initialize an empty list to store all fetched data
all_data = []

# Initialize the offset for the first request
offset = 0

# Iterate until all rows are fetched
while len(all_data) < total_rows:
    # Construct the request parameters
    params = {
        "$limit": rows_per_request,
        "$offset": offset
    }

    # Make the API request
    response = requests.get(url, params=params)

    # Check if the request was successful
    if response.status_code == 200:
        # Extract the data from the response
        data = response.json()

        # Append the fetched data to the list
        all_data.extend(data)

        # Update the offset for the next request
        offset += rows_per_request

        # Print the number of rows fetched so far
        print(f"Fetched {len(all_data)} rows")
    else:
        print(f"Error: {response.status_code}")
        break

# Open a CSV file for writing
with open("Food_Inspections.csv", "w", newline="", encoding="utf-8") as csvfile:
    # Get the header row from the first item in the list
    fieldnames = list(all_data[0].keys())

    # Create a CSV writer object
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    # Write the header row
    writer.writeheader()

    # Write each row of data to the CSV file
    for row in all_data:
        writer.writerow(row)

# Print the total number of rows fetched
print(f"Total rows fetched: {len(all_data)}")

Fetched 1000 rows
Fetched 2000 rows
Fetched 3000 rows
Fetched 4000 rows
Fetched 5000 rows
Fetched 6000 rows
Fetched 7000 rows
Fetched 8000 rows
Fetched 9000 rows
Fetched 10000 rows
Fetched 11000 rows
Fetched 12000 rows
Fetched 13000 rows
Fetched 14000 rows
Fetched 15000 rows
Fetched 16000 rows
Fetched 17000 rows
Fetched 18000 rows
Fetched 19000 rows
Fetched 20000 rows
Total rows fetched: 20000


### Load the data

In [4]:
df = pd.read_csv('Food_Inspections.csv')

df.rename({'inspection_id': 'Inspection ID', 'dba_name': 'DBA Name',
           'aka_name': 'AKA Name',
           'license_': 'License #',
           'facility_type': 'Facility Type',
           'risk': 'Risk',
           'address': 'Address',
           'city': 'City',
           'state': 'State',
           'zip': 'Zip',
           'inspection_date': 'Inspection Date',
           'inspection_type': 'Inspection Type',
           'results': 'Results',
           'violations': 'Violations',
           'latitude': 'Latitude',
           'longitude': 'Longitude',
           'location': 'Location',

}, axis='columns', inplace=True)

In [5]:
df.head()

,Inspection ID,DBA Name,AKA Name,License #,Facility Type,Risk,Address,City,State,Zip,...,Results,Violations,Latitude,Longitude,Location,:@computed_region_awaf_s7ux,:@computed_region_6mkv_f3dw,:@computed_region_vrxf_vc4k,:@computed_region_bdys_3d7i,:@computed_region_43wa_7qmu
0,2592528,"NORTHWEST INSTITUTE FOR CONTEMPORARY LEARNING,INC","NORTHWEST INSTITUTE FOR CONTEMPORARY LEARNING,INC",2475976.0,Children's Services Facility,Risk 1 (High),4040 W DIVISION ST,CHICAGO,IL,60651.0,...,Pass w/ Conditions,2. CITY OF CHICAGO FOOD SERVICE SANITATION CER...,41.902754,-87.728080,"{'latitude': '41.90275373866579', 'longitude':...",41.0,4299.0,24.0,286.0,45.0
1,2592545,CHICAGO PICKLE EATERY,CHICAGO PICKLE EATERY,2961426.0,Restaurant,Risk 1 (High),4515 N SHERIDAN RD,CHICAGO,IL,60640.0,...,Pass w/ Conditions,"3. MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL E...",41.964130,-87.654626,"{'latitude': '41.96413021890814', 'longitude':...",37.0,22616.0,31.0,20.0,39.0
2,2592557,NORTH BUENA DELI & WINE,NORTH BUENA DELI & WINE,2961965.0,Restaurant,Risk 1 (High),345 W ARMITAGE AVE,CHICAGO,IL,60614.0,...,Fail,"3. MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL E...",41.918221,-87.638252,"{'latitude': '41.918220945312974', 'longitude'...",51.0,21190.0,68.0,271.0,34.0
3,2592566,THE NEW VALOIS REST INC,THE NEW VALOIS RESTAURANT,3705.0,Restaurant,Risk 1 (High),1518 E 53RD ST,CHICAGO,IL,60615.0,...,Pass,"44. UTENSILS, EQUIPMENT & LINENS: PROPERLY STO...",41.799660,-87.588342,"{'latitude': '41.799659899582544', 'longitude'...",1.0,21192.0,8.0,500.0,10.0
4,2592546,Food 4 Less Midwest #552,FOOD 4 LESS,1596210.0,Grocery Store,Risk 1 (High),7030 S ASHLAND AVE,CHICAGO,IL,60636.0,...,Fail,"1. PERSON IN CHARGE PRESENT, DEMONSTRATES KNOW...",41.765791,-87.663967,"{'latitude': '41.76579077043187', 'longitude':...",17.0,22257.0,65.0,283.0,31.0


In [6]:
df.Results.unique()

array(['Pass w/ Conditions', 'Fail', 'Pass', 'Out of Business',
       'No Entry', 'Not Ready', 'Business Not Located'], dtype=object)

### Extract the columns of inspection date and inspection results

In [7]:
df['Inspection Date'] = pd.to_datetime(df['Inspection Date'])

In [8]:
results = df[['Inspection Date','Results']]

In [9]:
results.set_index('Inspection Date',inplace=True)

In [10]:
results.head()

,Results
Inspection Date,
2024-04-11,Pass w/ Conditions
2024-04-11,Pass w/ Conditions
2024-04-11,Fail
2024-04-11,Pass
2024-04-11,Fail


### Count the number of different results in the inspection
1. Create a dataframe **diff_results** from **results** and set the columns by different inspection results.
2. All the missing data are set as 0.
3. Combine the columns **Business Not Located**, **No Entry**, **Not Ready**, **Out of Business** into one column **Not Inspected**.

In [11]:
diff_results = results.pivot_table(index='Inspection Date',columns='Results',aggfunc=len)
diff_results = diff_results.replace(np.nan,0)

In [12]:
diff_results['Not Inspected']=diff_results[['Business Not Located','No Entry','Not Ready','Out of Business']].sum(axis=1)
col =['Pass','Pass w/ Conditions','Fail','Not Inspected']
diff_results = diff_results[col]

In [13]:
diff_results

Results,Pass,Pass w/ Conditions,Fail,Not Inspected
Inspection Date,,,,
2023-03-07,27.0,7.0,15.0,13.0
2023-03-08,41.0,14.0,14.0,11.0
2023-03-09,49.0,11.0,14.0,11.0
2023-03-10,35.0,13.0,21.0,10.0
2023-03-13,29.0,13.0,14.0,11.0
...,...,...,...,...
2024-04-05,33.0,7.0,10.0,10.0
2024-04-08,27.0,5.0,15.0,7.0
2024-04-09,27.0,12.0,14.0,9.0


### Get different inspection results in 2023

In [14]:
year_2023 = diff_results[diff_results.index.year == 2023]

In [15]:
year_2023

Results,Pass,Pass w/ Conditions,Fail,Not Inspected
Inspection Date,,,,
2023-03-07,27.0,7.0,15.0,13.0
2023-03-08,41.0,14.0,14.0,11.0
2023-03-09,49.0,11.0,14.0,11.0
2023-03-10,35.0,13.0,21.0,10.0
2023-03-13,29.0,13.0,14.0,11.0
...,...,...,...,...
2023-12-22,14.0,1.0,6.0,17.0
2023-12-26,17.0,4.0,6.0,6.0
2023-12-27,22.0,8.0,2.0,16.0


### Get different monthly inspection results in 2023
Creat a dataframe **year_2023_byMonth** to indicate the number inspection of each month in 2023

In [16]:
year_2023_byMonth = year_2023.groupby(by=year_2023.index.month).sum()
year_2023_byMonth

Results,Pass,Pass w/ Conditions,Fail,Not Inspected
Inspection Date,,,,
3,723.0,204.0,309.0,202.0
4,790.0,220.0,338.0,239.0
5,756.0,172.0,298.0,209.0
6,741.0,256.0,342.0,193.0
7,581.0,161.0,262.0,154.0
8,844.0,230.0,416.0,251.0
9,810.0,176.0,338.0,211.0
10,926.0,203.0,384.0,281.0
11,854.0,172.0,318.0,302.0


## Bar plot for the inspection number for a year

Create an interactive bar plot to show the number of different inspection results at a selected year. The plot should show different inspection results along the x-axis in a year and a bar showing the total number of inspections of each result type. You could use either a slider or a dropdown to choose the year.